In [1]:
import pandas as pd
import numpy as np
from EIMTC.preprocessing import OneHotEncoderEIMTC, M1CNNPreprocessing
from EIMTC.selection import train_test_split
from EIMTC.metrics import classification_report
from EIMTC.models import M1CNN

In [2]:
filepath = './data/iscx2016_merged.csv'
df = pd.read_csv(filepath, usecols=['udps.n_bytes', 'encapsulation', 'traffic_type'])
df

,udps.n_bytes,encapsulation,traffic_type
0,"[80, 79, 83, 84, 32, 47, 111, 99, 115, 112, 32...",nonvpn,video
1,"[23, 3, 3, 0, 58, 0, 0, 0, 0, 0, 0, 0, 7, 210,...",nonvpn,video
2,"[230, 107, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 11...",nonvpn,video
3,"[198, 218, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 11...",nonvpn,video
4,"[166, 113, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 11...",nonvpn,video
...,...,...,...
313862,"[43, 100, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 115...",nonvpn,video
313863,"[43, 100, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 115...",nonvpn,video
313864,"[68, 223, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 115...",nonvpn,video
313865,"[68, 223, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 115...",nonvpn,video


In [3]:
def preprocessing(df):
    df.dropna(inplace=True)
    M1CNNPreprocessing.preprocess_features(dataframe=df)
    enc = OneHotEncoderEIMTC()
    df['encapsulation_ohc'] = list(enc.fit_transform(df['encapsulation']).toarray())
    return df

In [4]:
df = preprocessing(df)
df

,udps.n_bytes,encapsulation,traffic_type,encapsulation_ohc
0,"[80, 79, 83, 84, 32, 47, 111, 99, 115, 112, 32...",nonvpn,video,"[1.0, 0.0]"
1,"[23, 3, 3, 0, 58, 0, 0, 0, 0, 0, 0, 0, 7, 210,...",nonvpn,video,"[1.0, 0.0]"
2,"[230, 107, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 11...",nonvpn,video,"[1.0, 0.0]"
3,"[198, 218, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 11...",nonvpn,video,"[1.0, 0.0]"
4,"[166, 113, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 11...",nonvpn,video,"[1.0, 0.0]"
...,...,...,...,...
313862,"[43, 100, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 115...",nonvpn,video,"[1.0, 0.0]"
313863,"[43, 100, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 115...",nonvpn,video,"[1.0, 0.0]"
313864,"[68, 223, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 115...",nonvpn,video,"[1.0, 0.0]"
313865,"[68, 223, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 115...",nonvpn,video,"[1.0, 0.0]"


In [5]:
n_bytes = len(df['udps.n_bytes'].iloc[0])
n_classes = len(df['encapsulation'].unique())
model = M1CNN(payload_size=n_bytes, n_classes=n_classes)

print(n_bytes, n_classes)

784 2


In [6]:
x_train, x_test, y_train, y_test = train_test_split(df['udps.n_bytes'].values, df['encapsulation_ohc'].values, 
    test_size=0.25,
    stratify=df['encapsulation'].values,
    random_state=42)

In [12]:
epochs = 10
batch_size = 128
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy'
)
model.fit(
    np.stack(x_train), 
    np.stack(y_train), 
    epochs=epochs, 
    batch_size=batch_size, 
    use_multiprocessing=True,
    workers=4,
    verbose=1
)

Epoch 1/10
1840/1840 [==============================] - 132s 69ms/step - loss: 0.4010 - categorical_accuracy: 0.9561 - recall_1: 0.9560 - precision_1: 0.9561: 1:26 - loss: 0.9999 - categorical_accuracy: 0.9469 - recall_1: 0.9465 - precis - ETA: 1 - ETA: 31s - 
Epoch 2/10
1840/1840 [==============================] - 135s 74ms/step - loss: 0.0901 - categorical_accuracy: 0.9633 - recall_1: 0.9633 - precision_1: 0.9633
Epoch 3/10
1840/1840 [==============================] - 130s 71ms/step - loss: 0.0889 - categorical_accuracy: 0.9653 - recall_1: 0.9653 - precision_1: 0.9653
Epoch 4/10
1840/1840 [==============================] - 114s 62ms/step - loss: 0.0744 - categorical_accuracy: 0.9755 - recall_1: 0.9755 - precision_1: 0.975512s - loss: 0.0749 - categorical_ac
Epoch 5/10
1840/1840 [==============================] - 132s 72ms/step - loss: 0.0588 - categorical_accuracy: 0.9827 - recall_1: 0.9827 - precision_1: 0.982711s - loss: 0.0589 - categorical_accuracy: 0.982
Epoch 6/10
1840/1840 [==

In [13]:
predictions = model.model.predict(np.stack(x_test))
predictions = np.argmax(predictions, axis=1)
y_test_true = np.argmax(np.stack(y_test), axis=1)

report = classification_report(y_test_true, predictions)
report

'              precision    recall  f1-score   support\n\n           0       0.99      1.00      0.99     73830\n           1       0.92      0.83      0.87      4637\n\n    accuracy                           0.99     78467\n   macro avg       0.96      0.91      0.93     78467\nweighted avg       0.99      0.99      0.99     78467\n'

In [14]:
with open('m1cnn_iscx2016_encaps_report.txt', "w+") as f:
    f.write(report)
